## Celda 1: Importaciones y Configuración Inicial del Logging


2025-05-30 15:42:55 - INFO - Agent GOSOM MVP Initialized (inicio de sesión de logger).
2025-05-30 15:42:55 - INFO - [ OK ] Archivo de configuración 'c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\config\parameters_default.json' cargado correctamente.
2025-05-30 15:42:55 - INFO - 
CARGANDO PARÁMETROS GLOBALES DEL AGENTE
2025-05-30 15:42:55 - INFO - [INFO] Lenguaje para scraping: es
2025-05-30 15:42:55 - INFO - [INFO] Profundidad de búsqueda por defecto: 2
2025-05-30 15:42:55 - INFO - [INFO] Prefijo para archivos de resultados: gmaps_data_
2025-05-30 15:42:55 - INFO - [INFO] Carpeta para CSVs crudos: c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\data\raw
2025-05-30 15:42:55 - INFO - [INFO] Carpeta para CSVs procesados: c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\data\processed
2025-05-30 15:42:55 - INFO - [ OK ] Carpetas de datos RAW y PROCESSED verificadas/creadas.
2025-05-30 15:42:55 - INFO - [INFO] Coordenadas cargadas para 4 c

[DEBUG RUTA] Directorio actual del notebook (CWD): c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\notebooks
[DEBUG RUTA] Directorio del agente GOSOM deducido: c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM
[DEBUG RUTA] Intentando cargar config desde: c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\config\parameters_default.json

        ********************************************************************************
        *                     G M A P S   S C R A P E R   A G E N T                    *
        *                         ( Avalian Project - MVP )                          *
        ********************************************************************************
        


## Celda 2 (Revisada): Funciones para Leer Archivos de Keywords


In [2]:
# Celda 2: Funciones para Leer Archivos de Keywords

logger.section("Definiendo Funciones de Utilidad")
logger.subsection("Función para Cargar Keywords")

def load_keywords_from_csv(city_name_key):
    """
    Carga keywords desde un archivo CSV específico para una clave de ciudad.
    La clave de ciudad se usa para encontrar el archivo en 'config/keywords_<city_name_key>.csv'
    """
    if CONFIG_DIR is None:
        logger.error("Directorio de configuración (CONFIG_DIR) no está definido. No se pueden cargar keywords.")
        return []

    filepath = os.path.join(CONFIG_DIR, f'keywords_{city_name_key.lower()}.csv')
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            keywords = [line.strip() for line in f if line.strip()] # Filtra líneas vacías y quita espacios
        
        if keywords:
            logger.success(f"Keywords cargadas para '{city_name_key}' desde '{filepath}': {len(keywords)} keywords.")
        else:
            logger.warning(f"No se encontraron keywords válidas (o el archivo está vacío) en '{filepath}' para '{city_name_key}'.")
        return keywords
    except FileNotFoundError:
        logger.error(f"Archivo de keywords no encontrado para '{city_name_key}' en '{filepath}'.")
        return []
    except Exception as e:
        logger.error(f"Error al cargar keywords para '{city_name_key}' desde '{filepath}': {e}", exc_info=True)
        return []

# --- Pruebas de la función load_keywords_from_csv (Opcional) ---
logger.subsection("Probando Carga de Keywords")

# Prueba con una ciudad que debería tener archivo de keywords
test_city_existing = 'neuquen' # Asegúrate de tener '0_AGENTE_GOSOM/config/keywords_neuquen.csv'
logger.info(f"Intentando cargar keywords para la ciudad de prueba: '{test_city_existing}'")
keywords_test_existing = load_keywords_from_csv(test_city_existing)

if keywords_test_existing:
    logger.info(f"Ejemplo de keywords para {test_city_existing.capitalize()} (primeras 3 si hay): {keywords_test_existing[:3]}")
else:
    # El warning o error ya fue logueado por la función load_keywords_from_csv
    pass

# Prueba con una ciudad que NO debería tener archivo de keywords
test_city_non_existing = 'ciudad_totalmente_inventada'
logger.info(f"Intentando cargar keywords para una ciudad de prueba que no existe: '{test_city_non_existing}'")
keywords_test_non_existing = load_keywords_from_csv(test_city_non_existing)
# (Se espera un error en el log si el archivo no existe, y la función devuelve [])

logger.info("Celda 2 completada: Función de carga de keywords definida y probada.")

2025-05-30 15:42:55 - INFO - 
DEFINIENDO FUNCIONES DE UTILIDAD
2025-05-30 15:42:55 - INFO - 
------------------------------------------------------------
Función para Cargar Keywords
------------------------------------------------------------
2025-05-30 15:42:55 - INFO - 
------------------------------------------------------------
Probando Carga de Keywords
------------------------------------------------------------
2025-05-30 15:42:55 - INFO - [INFO] Intentando cargar keywords para la ciudad de prueba: 'neuquen'
2025-05-30 15:42:55 - INFO - [ OK ] Keywords cargadas para 'neuquen' desde 'c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\config\keywords_neuquen.csv': 4 keywords.
2025-05-30 15:42:55 - INFO - [INFO] Ejemplo de keywords para Neuquen (primeras 3 si hay): ['Contadores en Neuquen', 'Empresas de servicios en Neuquen', 'Consultorios medicos en Neuquen']
2025-05-30 15:42:55 - INFO - [INFO] Intentando cargar keywords para una ciudad de prueba que no existe: 'ciu

## Celda 3 (Completa y Arreglada): Función para Ejecutar el Scraper GOSOM


2025-05-30 15:42:55 - INFO - 
DEFINIENDO FUNCIÓN PARA EJECUTAR GOSOM SCRAPER VÍA DOCKER


In [4]:
# --- Prueba de la función run_gmaps_scraper_docker ---
# Descomenta CON CUIDADO para probar una ejecución real. Asegúrate de que Docker Desktop esté corriendo.
# Y que tengas 'keywords_neuquen.csv' en '0_AGENTE_GOSOM/config/'
# Y que 'neuquen' esté definido en GMAPS_COORDINATES en 'parameters_default.json'

In [5]:
# --- Prueba de la función run_gmaps_scraper_docker ---
logger.section("Probando Ejecución del Scraper GOSOM para Neuquén")
test_city_for_scraping = 'neuquen'
keywords_for_scraping_test = load_keywords_from_csv(test_city_for_scraping) # Esto cargará las nuevas keywords que añadiste

if keywords_for_scraping_test and GMAPS_COORDINATES.get(test_city_for_scraping.lower()):
    logger.info(f"Iniciando prueba de scraping real para: {test_city_for_scraping.capitalize()} con {len(keywords_for_scraping_test)} keywords.")
    # Usar una profundidad baja para la prueba
    results_file = run_gmaps_scraper_docker(keywords_for_scraping_test, test_city_for_scraping, depth_override=1) 
    if results_file:
        logger.success(f"PRUEBA DE SCRAPING FINALIZADA. Archivo generado: {results_file}")
        try:
            if os.path.getsize(results_file) > 0:
                logger.info(f"El archivo de resultados '{results_file}' tiene contenido (tamaño: {os.path.getsize(results_file)} bytes).")
                # Puedes descomentar lo siguiente para ver las primeras filas si tienes gmaps_column_names ya definida
                # (pero se define formalmente en Celda 5, así que cuidado con errores de variable no definida aquí)
                # df_test_results = pd.read_csv(results_file, header=None, names=gmaps_column_names, nrows=2)
                # display(df_test_results)
            else:
                logger.warning(f"El archivo de resultados '{results_file}' está vacío.")
        except NameError:
             logger.warning("Variable 'gmaps_column_names' no definida aún para mostrar preview del CSV en Celda 3.")
        except Exception as e_read:
            logger.error(f"No se pudo leer/verificar el archivo de resultados de prueba: {e_read}", exc_info=True)
    else:
        logger.error(f"PRUEBA DE SCRAPING FALLÓ para {test_city_for_scraping.capitalize()}. Revisa los logs.")
else:
    logger.warning(f"No se ejecutará la prueba de scraping. Faltan keywords o coordenadas para '{test_city_for_scraping}'.")

2025-05-30 15:42:55 - INFO - 
PROBANDO EJECUCIÓN DEL SCRAPER GOSOM PARA NEUQUÉN
2025-05-30 15:42:55 - INFO - [ OK ] Keywords cargadas para 'neuquen' desde 'c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\config\keywords_neuquen.csv': 4 keywords.
2025-05-30 15:42:55 - INFO - [INFO] Iniciando prueba de scraping real para: Neuquen con 4 keywords.
2025-05-30 15:42:55 - INFO - 
------------------------------------------------------------
Iniciando scraping para ciudad: 'Neuquen'
------------------------------------------------------------
2025-05-30 15:42:55 - INFO - [INFO] Usando profundidad de búsqueda: 1
2025-05-30 15:42:55 - INFO - [INFO] Creando archivo temporal de queries en: c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-GMAPS\etl-gmaps\0_AGENTE_GOSOM\config\temp_queries_neuquen_20250530_154255.txt
2025-05-30 15:42:55 - INFO - [ OK ] Archivo temporal de queries creado con 4 keywords.
2025-05-30 15:42:55 - INFO - [INFO] Creando archivo de salida CSV vacío en: c:\BAU\1 TRABAJO\REPO

## Celda 4: Orquestación del Proceso de Scraping para Ciudades Definidas


Esta celda utilizará las funciones de las celdas anteriores para iterar sobre una lista de ciudades (o todas las que tengan keywords y coordenadas), ejecutar el scraper para cada una, y recolectar las rutas a los archivos CSV crudos generados.

In [6]:
# Celda 4: Orquestación del Proceso de Scraping

logger.section("Orquestando Tareas de Scraping")

# Lista de ciudades a procesar. Podrías obtenerlas de las claves en GMAPS_COORDINATES
# o tener una lista explícita.
# Por ahora, procesaremos solo las ciudades para las que tengamos archivos de keywords y coordenadas.

cities_to_process = []
if GMAPS_COORDINATES and CONFIG_DIR:
    for city_key in GMAPS_COORDINATES.keys():
        # Verificar si existe un archivo de keywords para esta ciudad
        keywords_file_path = os.path.join(CONFIG_DIR, f'keywords_{city_key.lower()}.csv')
        if os.path.exists(keywords_file_path):
            cities_to_process.append(city_key)
            logger.info(f"Ciudad '{city_key.capitalize()}' añadida a la lista de procesamiento (tiene coordenadas y archivo de keywords).")
        else:
            logger.warning(f"No se encontró archivo de keywords para '{city_key}' en '{keywords_file_path}'. Se omitirá esta ciudad.")
else:
    logger.error("No hay coordenadas (GMAPS_COORDINATES) o directorio de configuración (CONFIG_DIR) definido. No se pueden determinar ciudades a procesar.")


raw_csv_files_generated = {} # Diccionario para guardar: {'ciudad': 'ruta_al_csv_crudo'}

if not cities_to_process:
    logger.warning("No hay ciudades configuradas para procesar. Revisa GMAPS_COORDINATES y los archivos 'keywords_ciudad.csv'.")
else:
    logger.info(f"Ciudades a procesar en esta ejecución: {', '.join([c.capitalize() for c in cities_to_process])}")
    
    for city_key in cities_to_process:
        logger.subsection(f"Procesando Ciudad: {city_key.capitalize()}")
        
        keywords = load_keywords_from_csv(city_key)
        if not keywords:
            logger.warning(f"No se cargaron keywords para {city_key.capitalize()}, se omite el scraping para esta ciudad.")
            continue

        # Aquí puedes decidir la profundidad. Podrías tenerla en GMAPS_COORDINATES por ciudad,
        # o usar un valor diferente para pruebas.
        # depth_for_city = GMAPS_COORDINATES[city_key].get('depth', DEFAULT_DEPTH) 
        depth_for_city = 1 # Para el MVP y pruebas rápidas, usar una profundidad baja.
        
        logger.info(f"Intentando scraping para {city_key.capitalize()} con {len(keywords)} keywords y profundidad {depth_for_city}.")
        
        # --- ESTA LÍNEA EJECUTARÁ DOCKER ---
        # (En la prueba real, esto puede tardar varios minutos por ciudad)
        # raw_file_path = run_gmaps_scraper_docker(keywords, city_key, depth_override=depth_for_city)
        # --- FIN DE LÍNEA QUE EJECUTA DOCKER ---

        # --- PARA DESARROLLO SIN EJECUTAR DOCKER CADA VEZ ---
        # Simula que el scraper se ejecutó y generó un archivo.
        # Asegúrate de tener un archivo de ejemplo en data/raw/ para que esto funcione sin Docker.
        # Por ejemplo: data/raw/gmaps_data_neuquen_dummy.csv
        logger.warning("MODO SIMULACIÓN: `run_gmaps_scraper_docker` está COMENTADO. Usando archivo dummy si existe.")
        dummy_file_name = f"{RESULTS_FILENAME_PREFIX}{city_key.lower()}_dummy.csv"
        dummy_file_path = os.path.join(RAW_CSV_FOLDER, dummy_file_name)
        if os.path.exists(dummy_file_path):
            raw_file_path = dummy_file_path
            logger.info(f"MODO SIMULACIÓN: Usando archivo dummy existente: {raw_file_path}")
        else:
            raw_file_path = None
            logger.warning(f"MODO SIMULACIÓN: Archivo dummy NO encontrado en {dummy_file_path}. No habrá datos para esta ciudad en simulación.")
        # --- FIN DE SECCIÓN DE SIMULACIÓN ---

        if raw_file_path and os.path.exists(raw_file_path):
            raw_csv_files_generated[city_key] = raw_file_path
            logger.success(f"Scraping para {city_key.capitalize()} simulado/completado. Archivo: {raw_file_path}")
        else:
            logger.error(f"Scraping para {city_key.capitalize()} falló o no generó archivo (o archivo dummy no encontrado).")

if raw_csv_files_generated:
    logger.success(f"Proceso de scraping (o simulación) finalizado. Archivos crudos generados:")
    for city, path in raw_csv_files_generated.items():
        logger.info(f"  - {city.capitalize()}: {path}")
else:
    logger.warning("No se generaron archivos CSV crudos en esta ejecución.")

logger.info("Celda 4 completada: Orquestación de scraping (o simulación) definida.")

2025-05-30 15:42:56 - INFO - 
ORQUESTANDO TAREAS DE SCRAPING
2025-05-30 15:42:56 - INFO - [INFO] Ciudad 'Neuquen' añadida a la lista de procesamiento (tiene coordenadas y archivo de keywords).
2025-05-30 15:42:56 - INFO - [INFO] Ciudad 'General_roca' añadida a la lista de procesamiento (tiene coordenadas y archivo de keywords).
2025-05-30 15:42:56 - INFO - [INFO] Ciudad 'Cipolletti' añadida a la lista de procesamiento (tiene coordenadas y archivo de keywords).
2025-05-30 15:42:56 - INFO - [INFO] Ciudad 'Allen' añadida a la lista de procesamiento (tiene coordenadas y archivo de keywords).
2025-05-30 15:42:56 - INFO - [INFO] Ciudades a procesar en esta ejecución: Neuquen, General_roca, Cipolletti, Allen
2025-05-30 15:42:56 - INFO - 
------------------------------------------------------------
Procesando Ciudad: Neuquen
------------------------------------------------------------
2025-05-30 15:42:56 - INFO - [ OK ] Keywords cargadas para 'neuquen' desde 'c:\BAU\1 TRABAJO\REPOSITORIOS\ETL-

## Celda 5: Carga y Transformación de Datos


2025-05-30 15:42:56 - INFO - 
CARGANDO Y TRANSFORMANDO DATOS EXTRAÍDOS
2025-05-30 15:42:56 - WARNING - [WARN] No hay archivos CSV crudos para procesar (variable 'raw_csv_files_generated' está vacía).
2025-05-30 15:42:56 - WARNING - [WARN] No se procesaron datos de ninguna ciudad. El DataFrame final está vacío.
2025-05-30 15:42:56 - INFO - [INFO] Celda 5 completada: Carga y transformación de datos definida.


In [8]:
# Ejemplo de deduplicación (añadir después de concatenar en Celda 5)
if not final_df_all_cities.empty:
    logger.subsection("Aplicando Deduplicación de Prospectos")
    # Deduplicar basado en el link de Google Maps, manteniendo la primera aparición
    # Podrías necesitar una lógica más sofisticada si los links no son siempre perfectos o si quieres fusionar info
    initial_count = len(final_df_all_cities)
    final_df_all_cities.drop_duplicates(subset=['link'], keep='first', inplace=True)
    deduplicated_count = initial_count - len(final_df_all_cities)
    if deduplicated_count > 0:
        logger.success(f"Se eliminaron {deduplicated_count} prospectos duplicados basados en la columna 'link'.")
    else:
        logger.info("No se encontraron duplicados basados en la columna 'link' o ya estaban limpios.")
    logger.info(f"Total de prospectos únicos después de deduplicación: {len(final_df_all_cities)}")


## Celda 6: Guardado de Datos Procesados (a un único CSV por ahora)


In [9]:
# Celda 6: Guardado de Datos Procesados

logger.section("Guardando Datos Procesados y Limpios")

if not final_df_all_cities.empty:
    timestamp_save = datetime.now().strftime("%Y%m%d_%H%M%S")
    processed_filename = f"gmaps_prospectos_consolidados_{timestamp_save}.csv"
    
    if PROCESSED_CSV_FOLDER: # Verificar que la carpeta de destino esté definida
        processed_filepath = os.path.join(PROCESSED_CSV_FOLDER, processed_filename)
        try:
            final_df_all_cities.to_csv(processed_filepath, index=False, encoding='utf-8-sig') # utf-8-sig para mejor compatibilidad con Excel
            logger.success(f"Datos procesados consolidados guardados en: {processed_filepath}")
            logger.info(f"Total de {len(final_df_all_cities)} prospectos guardados.")
        except Exception as e:
            logger.error(f"Error al guardar el archivo CSV procesado en '{processed_filepath}': {e}", exc_info=True)
    else:
        logger.error("PROCESSED_CSV_FOLDER no está definido. No se puede guardar el archivo procesado.")
elif raw_csv_files_generated and not all_processed_data : # Hubo archivos crudos pero no se procesó nada
    logger.warning("Se generaron archivos crudos pero no se pudieron procesar o resultaron vacíos. No se guardará archivo consolidado.")
else: # No hubo archivos crudos para empezar
    logger.warning("No hay datos procesados para guardar (DataFrame final está vacío).")

logger.info("Celda 6 completada: Proceso de guardado de datos procesados definido.")

2025-05-30 15:42:56 - INFO - 
GUARDANDO DATOS PROCESADOS Y LIMPIOS
2025-05-30 15:42:56 - WARNING - [WARN] No hay datos procesados para guardar (DataFrame final está vacío).
2025-05-30 15:42:56 - INFO - [INFO] Celda 6 completada: Proceso de guardado de datos procesados definido.
